Import libraries and data

In [11]:
import pandas as pd
import numpy as np

In [1]:


links = pd.read_csv('ml-latest-small/links.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [32]:
# display all columns and rows
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [10]:
users_items = pd.pivot_table(data=ratings, 
                                 values='rating', 
                                 index='userId', 
                                 columns='movieId')

users_items = users_items.fillna(0)
users_items

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Create a DataFrame that contains the positions of all nonzero ratings.
# This will be used to split the data into training and test sets.
ratings_pos = pd.DataFrame(
    np.nonzero(np.array(users_items)),
).T
ratings_pos

,0,1
0,0,0
1,0,2
2,0,5
3,0,43
4,0,46
...,...,...
100831,609,9416
100832,609,9443
100833,609,9444
100834,609,9445


In [13]:
# rename the columns
ratings_pos.columns = ['row_pos', 'column_pos']
ratings_pos

,row_pos,column_pos
0,0,0
1,0,2
2,0,5
3,0,43
4,0,46
...,...,...
100831,609,9416
100832,609,9443
100833,609,9444
100834,609,9445


In [15]:
# Get the nonzero ratings from the positions above.
users_items.iloc[0:1]

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# this is numpy array of all ratings in the dataset (all users and all items)
ratings_vals = np.array(users_items)[ratings_pos['row_pos'], ratings_pos['column_pos']]
ratings_vals

array([4., 4., 4., ..., 5., 5., 3.])

In [20]:
len(ratings_vals)

100836

Train and test split

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
# Split nonzero ratings into train and test sets.
train_pos, test_pos = train_test_split(ratings_pos, random_state=123, test_size=.1)

In [23]:
train_pos.sort_values(['row_pos', 'column_pos']).head(3)

,row_pos,column_pos
0,0,0
1,0,2
3,0,43


In [24]:
test_pos.sort_values(['row_pos', 'column_pos']).head(3)

,row_pos,column_pos
2,0,5
43,0,632
54,0,787


Create the train DataFrame

In [25]:
# The train and test datasets will both have the same shape as the `users_items`DataFrame. Most of their values will be zero, except for the values in the positions stored inside the `train_pos` set for the train dataset,
# and the `test_pos`set for the test dataset.
# Create a copy of the users-items DataFrame and set all values to zero.
train = users_items.copy()

for column in train.columns:
    train[column].values[:] = 0

In [27]:
# Sum the values in the DataFrame to check that all of them are equal to zero.
train.sum().sum()

#alternative way
# train.values.sum()

0.0

In [28]:
# Iterate over the nonzero positions in the train dataset.
# Get the corresponding rating for each nonzero position from the users_items DataFrame.
# Insert that value into the newly created DataFrame at the same position.
for pos in train_pos.values:
    index = pos[0]
    col = pos[1]
    train.iloc[index, col] = users_items.iloc[index, col]

In [33]:
train.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,85,86,87,88,89,92,93,94,95,96,97,99,100,101,102,103,104,105,106,107,108,110,111,112,113,116,117,118,119,121,122,123,125,126,128,129,132,135,137,140,141,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,183,184,185,186,187,188,189,190,191,193,194,195,196,198,199,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,265,266,267,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,287,288,289,290,291,292,293,294,295,296,298,299,300,301,302,303,304,305,306,307,308,310,311,312,313,314,315,316,317,318,319,320,321,322,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,393,405,406,407,408,409,410,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,440,441,442,444,445,446,448,449,450,451,452,453,454,455,456,457,458,459,460,461,464,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,484,485,486,487,488,489,490,491,492,493,494,495,496,497,499,500,501,502,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,526,527,528,529,531,532,533,534,535,536,537,538,539,540,541,542,543,544,546,547,548,549,550,551,552,553,555,556,558,562,563,564,567,568,569,573,574,575,577,579,...,154975,155064,155168,155288,155358,155509,155589,155659,155743,155774,155812,155820,155892,156025,156371,156387,156553,156605,156607,156609,156675,156706,156726,156781,156783,157108,157110,157122,157130,157172,157200,157270,157296,157312,157340,157369,157407,157432,157699,157775,157865,158022,158027,158035,158238,158254,158388,158398,158402,158528,158721,158783,158813,158830,158842,158872,158874,158882,158956,158966,158972,159061,159069,159077,159093,159161,159193,159403,159415,159441,159510,159690,159717,159755,159779,159811,159817,159849,159858,159976,160080,160271,160289,160341,160400,160422,160438,160440,160527,160563,160565,160567,160569,160571,160573,160644,160646,160684,160718,160730,160836,160848,160872,160954,160978,160980,161008,161024,161032,161044,161127,161131,161290,161354,161580,161582,161594,161634,161830,161918,161922,161966,162082,162344,162350,162414,162478,162578,162590,162598,162600,162602,162606,162828,162968,162982,163056,163072,163112,163134,163386,163527,163639,163645,163653,163809,163925,163937,163981,163985,164179,164200,164226,164280,164367,164375,164540,164647,164655,164707,164753,164881,164909,164917,165075,165101,165103,165139,165343,165347,165483,165489,165529,165549,165551,165635,165639,165645,165671,165843,165947,165959,165969,166015,166024,166183,166203,166291,166461,166492,166526,166528,166534,166558,166568,166635,166643,166705,166946,167018,167036,167064,167296,167370,167380,167538,167570,167634,167706,167732,167746,167772,167790,167854,168026,168090,168144,168174,168218,168248,168250,168252,168254,168266,168326,168350,168358,168366,168418,168456,168492,168608,168612,168632,168712,168846,169034,169180,169670,169904,169912,169958,169982,169984,169992,170289,170297,170355,170357,170399,170401,170411,170551,170597,170697,170705,170777,170813,170817,170827,170837,170875,170897,170907,170937,170939,170945,170957,170993,171011,171023,171251,171495,171631,171695,171701,171749,171751,171759,171763,171765,171811,171867,171891,171917,172013,172215,172229,172233,172253,172321,172461,172497,172547,172577,172583,172585,172587

In [36]:
train.iloc[0:1]

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,85,86,87,88,89,92,93,94,95,96,97,99,100,101,102,103,104,105,106,107,108,110,111,112,113,116,117,118,119,121,122,123,125,126,128,129,132,135,137,140,141,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,183,184,185,186,187,188,189,190,191,193,194,195,196,198,199,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,222,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,265,266,267,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,287,288,289,290,291,292,293,294,295,296,298,299,300,301,302,303,304,305,306,307,308,310,311,312,313,314,315,316,317,318,319,320,321,322,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,393,405,406,407,408,409,410,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,440,441,442,444,445,446,448,449,450,451,452,453,454,455,456,457,458,459,460,461,464,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,484,485,486,487,488,489,490,491,492,493,494,495,496,497,499,500,501,502,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,526,527,528,529,531,532,533,534,535,536,537,538,539,540,541,542,543,544,546,547,548,549,550,551,552,553,555,556,558,562,563,564,567,568,569,573,574,575,577,579,...,154975,155064,155168,155288,155358,155509,155589,155659,155743,155774,155812,155820,155892,156025,156371,156387,156553,156605,156607,156609,156675,156706,156726,156781,156783,157108,157110,157122,157130,157172,157200,157270,157296,157312,157340,157369,157407,157432,157699,157775,157865,158022,158027,158035,158238,158254,158388,158398,158402,158528,158721,158783,158813,158830,158842,158872,158874,158882,158956,158966,158972,159061,159069,159077,159093,159161,159193,159403,159415,159441,159510,159690,159717,159755,159779,159811,159817,159849,159858,159976,160080,160271,160289,160341,160400,160422,160438,160440,160527,160563,160565,160567,160569,160571,160573,160644,160646,160684,160718,160730,160836,160848,160872,160954,160978,160980,161008,161024,161032,161044,161127,161131,161290,161354,161580,161582,161594,161634,161830,161918,161922,161966,162082,162344,162350,162414,162478,162578,162590,162598,162600,162602,162606,162828,162968,162982,163056,163072,163112,163134,163386,163527,163639,163645,163653,163809,163925,163937,163981,163985,164179,164200,164226,164280,164367,164375,164540,164647,164655,164707,164753,164881,164909,164917,165075,165101,165103,165139,165343,165347,165483,165489,165529,165549,165551,165635,165639,165645,165671,165843,165947,165959,165969,166015,166024,166183,166203,166291,166461,166492,166526,166528,166534,166558,166568,166635,166643,166705,166946,167018,167036,167064,167296,167370,167380,167538,167570,167634,167706,167732,167746,167772,167790,167854,168026,168090,168144,168174,168218,168248,168250,168252,168254,168266,168326,168350,168358,168366,168418,168456,168492,168608,168612,168632,168712,168846,169034,169180,169670,169904,169912,169958,169982,169984,169992,170289,170297,170355,170357,170399,170401,170411,170551,170597,170697,170705,170777,170813,170817,170827,170837,170875,170897,170907,170937,170939,170945,170957,170993,171011,171023,171251,171495,171631,171695,171701,171749,171751,171759,171763,171765,171811,171867,171891,171917,172013,172215,172229,172233,172253,172321,172461,172497,172547,172577,172583,172585,172587

Create the test DataFrame

In [37]:
# Create a copy of the users-items DataFrame and set all values to zero.
test = users_items.copy()

# Iterate over the nonzero positions in the test dataset.
# Get the corresponding rating for each nonzero position from the users_items DataFrame.
# Insert that value in the newly created DataFrame at the same position.
for column in test.columns:
    test[column].values[:] = 0

for pos in test_pos.values:
    index = [0]
    col = pos[1]
    test.iloc[index, col] = users_items.iloc[index, col]

In [39]:
test.iloc[0:1, [1, 2, 6, 47, 50, 101, 110, 151, 157, 163, 216]]

movieId,2,3,7,52,55,116,128,179,186,193,252
userId,,,,,,,,,,,
1,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
#build a compact DataFrame to store the positions of all the places in the test set and their true rating
true_test_ratings = []

# Iterate over rows and get the values in the two columns (= positions in users_items).
# Use positions to get ratings and store them in the true_test_ratings list.
for index, row in test_pos.iterrows():
    true_test_ratings.append(users_items.iloc[row[0], row[1]])


In [41]:
# Add ratings as new column
test_pos = test_pos.assign(true_rating = true_test_ratings)

In [44]:
test_pos.head()

,row_pos,column_pos,true_rating
16705,104,7306,5.0
3692,21,3012,4.5
74367,473,3578,3.0
49894,318,8945,4.5
84800,549,8532,4.5


Create the similarity matrix for the train set

In [46]:
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
# Get cosine similarities for the train dataset.
train_similarity = pd.DataFrame(cosine_similarity(train), columns=train.index,
                                index=train.index)
train_similarity.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.015234,0.067484,0.171337,0.128902,0.121473,0.131189,0.148958,0.072979,0.010804,0.132643,0.018261,0.082074,0.100530,0.127156,0.138167,0.234544,0.182769,0.289831,0.139626,0.127037,0.027735,0.092984,0.134795,0.081269,0.112933,0.216925,0.183690,0.144867,0.070312,0.120711,0.138929,0.150566,0.080602,0.083305,0.073148,0.156712,0.150578,0.297472,0.094617,0.098219,0.227871,0.116654,0.078794,0.306169,0.107648,0.064145,0.092707,0.092848,0.101356,0.108377,0.033357,0.07131,0.127218,0.059802,0.148391,0.308549,0.167701,0.200234,0.070655,0.107152,0.099661,0.187425,0.287393,0.071879,0.227027,0.120647,0.242680,0.134346,0.085793,0.085539,0.206424,0.043817,0.065664,0.130804,0.172800,0.073941,0.160360,0.170692,0.062165,0.112503,0.194837,0.102798,0.157011,0.016998,0.132148,0.034022,0.071163,0.024718,0.056793,0.298620,0.033093,0.230747,0.116522,0.150440,0.263278,0.080021,0.059412,0.075242,0.130345,0.131216,0.147261,0.202497,0.119851,0.086287,0.021847,0.054598,0.052841,0.156305,0.140034,0.061327,0.153907,0.074147,0.076704,0.176578,0.160980

In [ ]:
# Evaluate whether a recommender system using the sum of rating + food_rating + service_rating instead of only the rating is better?



In [ ]:
# to improve the accuracy of predictions, 
# 1. drop users with less than 10 (or other number) ratings
# 2. drop items with less than 10 (or other number) ratings
# 3. drop all 0 ratings
# 4. we can use the mean of the ratings for each user as the default value for the predictions.